# Figure 4.12 of Chapter 4
As with some previous figures, the infectious seed isn't specified
which is presumably the reason why the curves aren't absolutely identical.
Here, Vynnycky and White may not be using a numeric simulation,
but rather calculating the epidemic final size from an analytic expression.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def build_sir_model(
    model_config: dict,
) -> CompartmentalModel:
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    analysis_times = (
        model_config["start_time"], 
        model_config["end_time"],
    )
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": model_config["population"] - model_config["seed"], 
            "infectious": model_config["seed"],
        }
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"),
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery"),
        source="infectious", 
        dest="recovered",
    )
    return model

In [ ]:
parameters = {
    "recovery": 0.333,
}
r0s = np.linspace(1.1, 15., 100)
config = {
    "start_time": 0.,
    "end_time": 20.,
    "population": 1.,
    "seed": 0.01,
}

outputs = pd.DataFrame(columns=r0s)
sir_model = build_sir_model(config)

for r0 in r0s:
    parameters["contact_rate"] = r0 * parameters["recovery"]
    sir_model.run(parameters=parameters)
    outputs[r0] = 1. - sir_model.get_outputs_df()["susceptible"]

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))
left_ax.plot(
    r0s,
    outputs.iloc[-1],
    color="k",
)
left_ax.set_ylabel("Proportion infected by the end of the epidemic")
left_ax.set_ylim((0., 1.))
left_ax.set_xlabel("Basic reproduction number")
left_ax.set_xlim((0., 15.))
image = plt.imread("./figures/fig_4_12.jpg")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");